<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/04_CAG_GC.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 04. CAG GC

## Overview  
In this exercise, we will explore the concept of Credibility-Aware Generation (CAG) and Groundedness Check (GC) using the Solar framework. CAG focuses on generating responses that are credible and factually accurate, while GC ensures that the generated content is grounded in verifiable sources. This notebook will guide you through the implementation and application of these techniques to enhance the reliability and accuracy of the model's responses.
 
## Purpose of the Exercise
The purpose of this exercise is to demonstrate the practical application of Credibility-Aware Generation and Groundedness Check within the Solar framework. By the end of this tutorial, users will understand how to utilize these techniques to produce credible and well-grounded outputs from the language model, thus improving the trustworthiness and utility of the generated content.


In [79]:
! pip3 install -qU langchain-upstage python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [81]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [82]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

In [83]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer for the given question from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [84]:
context = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""

chain.invoke(
    {
        "question": "What is the license under which SOLAR 10.7B is available?",
        "Context": context,
    }
)

'The license under which SOLAR 10.7B is available is the Apache 2.0 license.'

In [85]:
chain.invoke({"question": "Who is the created SOLAR 10.7B?", "Context": context})

'The information is not present in the context.'

In [86]:
chain.invoke(
    {
        "question": "Did Google provide resources for the SOLAR 10.7B project?",
        "Context": context,
    }
)

'The information is not present in the context.'

In [87]:
chain.invoke({"question": "What does DUS stand for?", "Context": context})

'DUS stands for depth up-scaling.'

In [88]:
answer = chain.invoke(
    {
        "question": "What is the name of the variant fine-tuned for instruction-following capabilities?",
        "Context": context,
    }
)
print(answer)

SOLAR 10.7B-Instruct


## Groundedness Check with LangChain and Upstage
![Groundedness](./figures/gc.png)


In [91]:
# GC
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

answer = chain.invoke(
    {
        "question": "What is DUS?",
        "Context": context,
    }
)
print("Potential answer: ", answer)
gc_result = groundedness_check.invoke({"context": context, "answer": answer})

print("GC check result: ", gc_result)
if gc_result.lower().startswith("grounded"):
    print("✅ Groundedness check passed")
else:
    print("❌ Groundedness check failed")

Potential answer:  DUS stands for depth up-scaling.
GC check result:  grounded
✅ Groundedness check passed


In [92]:
# GC
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

answer = "Solar 10.7B is available to the public with a non-commercial license."
gc_result = groundedness_check.invoke({"context": context, "answer": answer})

print(gc_result)
if gc_result.lower().startswith("grounded"):
    print("✅ Groundedness check passed")
else:
    print("❌ Groundedness check failed")

notGrounded
❌ Groundedness check failed


# Excercise 

Write code to check GC and make LLM more secure. For example, check GC several times until LLM gives you a reliable answer.